In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.webdriver import WebDriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd 
import re

In [14]:
def team_data(lista, nazwa, data_for_dataframe):

    pattern = r'data-stat="([^"]+)">([^<]+)'

    matches = re.findall(pattern, lista)

    for match in matches:
        data_type, value = match
        data_for_dataframe[nazwa +data_type] = value


In [15]:
def getMatchDate(page_source):
    date_of_match = re.findall(r'data-venue-date=([^ ]+)', page_source)
    date_of_match = date_of_match[0].replace('"', '')

    return date_of_match

In [16]:
def getMatchTime(page_source):
    game_time = re.findall(r'data-venue-time=([^ ]+)', page_source)
    game_time = game_time[0].replace('"', '')

    return game_time

In [17]:
def getClubsNames(page_source):
    teams = re.findall(r'<h1>.*Match', page_source)
    teams = teams[0].replace(' Match', '')
    teams = teams.replace('<h1>','')

    teams_names = teams.split(" vs. ")

    return teams_names

In [18]:

def getData(page_source):

    data_for_dataframe = {}

    data = re.findall("<tfoot>.*<\/tfoot>", page_source)

    home_team = data[0:6]
    away_team = data[6:12]


    home_team_summary = home_team[0]
    home_team_passes = home_team[1]
    home_team_pass_type = home_team[2]
    home_team_defensive = home_team[3]
    home_team_possesion = home_team[4]
    home_team_miscellaneous = home_team[5]

    away_team_summary = away_team[0]
    away_team_passes = away_team[1]
    away_team_pass_type = away_team[2]
    away_team_defensive = away_team[3]
    away_team_possesion = away_team[4]
    away_team_miscellaneous = away_team[5]

    team_data(home_team_summary, 'home_team_summary_', data_for_dataframe)
    team_data(home_team_passes, 'home_team_passes_', data_for_dataframe)
    team_data(home_team_pass_type, 'home_team_pass_type_', data_for_dataframe)
    team_data(home_team_defensive, 'home_team_defensive_', data_for_dataframe)
    team_data(home_team_possesion, 'home_team_possesion_', data_for_dataframe)
    team_data(home_team_miscellaneous, 'home_team_miscellaneous_', data_for_dataframe)

    team_data(away_team_summary, 'away_team_summary_', data_for_dataframe)
    team_data(away_team_passes, 'away_team_passes_', data_for_dataframe)
    team_data(away_team_pass_type, 'away_team_pass_type_', data_for_dataframe)
    team_data(away_team_defensive, 'away_team_defensive_', data_for_dataframe)
    team_data(away_team_possesion, 'away_team_possesion_', data_for_dataframe)
    team_data(away_team_miscellaneous, 'away_team_miscellaneous_', data_for_dataframe)

    df = pd.DataFrame.from_dict(data_for_dataframe, orient ='index')

    df = df.T

    df['game_date'] = getMatchDate(page_source)
    df['game_time'] = getMatchTime(page_source)

    teams_names = getClubsNames(page_source)

    df['home_team'] = teams_names[0]
    df['away_team'] = teams_names[1]

    return df

In [7]:
def getDataFromPage(url):

    driver = webdriver.Chrome(executable_path='/Users/mikolajpaterka/local/bin/chromedriver')

    driver.get(url)

    page_source = driver.page_source

    driver.quit()

    return page_source

In [11]:
main_urls = ['https://fbref.com/en/comps/20/2021-2022/schedule/2021-2022-Bundesliga-Scores-and-Fixtures']
    
for main_url in main_urls:
    print(main_url)

    data = getDataFromPage(main_url)
    dataframe = getData(data)


/var/folders/_h/ljwht4gd7lb99rm1hm78h7_00000gn/T/ipykernel_51259/274481876.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/mikolajpaterka/local/bin/chromedriver')


In [12]:
dataframe

,home_team_summary_player,home_team_summary_minutes,home_team_summary_goals,home_team_summary_assists,home_team_summary_pens_made,home_team_summary_pens_att,home_team_summary_shots,home_team_summary_shots_on_target,home_team_summary_cards_yellow,home_team_summary_cards_red,...,away_team_miscellaneous_pens_conceded,away_team_miscellaneous_own_goals,away_team_miscellaneous_ball_recoveries,away_team_miscellaneous_aerials_won,away_team_miscellaneous_aerials_lost,away_team_miscellaneous_aerials_won_pct,game_date,game_time,home_team,away_team
0,16 Players,990,1,0,0,0,8,2,2,0,...,0,0,62,3,6,33.3,2022-08-05,20:30,Eintracht Frankfurt,Bayern Munich
